In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Filename

In [ ]:
name = 'file_0.csv'

## Libraries

In [2]:
!pip install cchardet &> /dev/null
!pip install requests-futures &> /dev/null
!pip install http_request_randomizer &> /dev/null

## Imports

In [3]:
from requests import Session
from requests_futures.sessions import FuturesSession
from bs4 import BeautifulSoup
from os import path, mkdir
import re
import pandas as pd
import time
import cchardet
import lxml
from numpy import random
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

# Scraping

In [4]:
req_proxy = RequestProxy()  
proxy_list = req_proxy.get_proxy_list()

2021-05-13 17:38:13,532 http_request_randomizer.requests.useragent.userAgent INFO     Using local file for user agents: /usr/local/lib/python3.7/dist-packages/http_request_randomizer/requests/proxy/../data/user_agents.txt
2021-05-13 17:38:13,536 root   DEBUG    === Initialized Proxy Parsers ===
2021-05-13 17:38:13,538 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2021-05-13 17:38:13,541 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2021-05-13 17:38:13,543 root   DEBUG    	 SslProxy parser of 'https://www.sslproxies.org' with required bandwidth: '150' KBs
2021-05-13 17:38:13,545 root   DEBUG    =================================
2021-05-13 17:38:13,787 root   DEBUG    Added 300 proxies from FreeProxy
2021-05-13 17:38:14,645 http_request_randomizer.requests.parsers.PremProxyParser DEBUG    Pages: {'', '06.htm', '03.htm', '04.htm', '05.htm', '09.htm', '02.htm', '07.htm', '08.htm

In [5]:
table = pd.read_csv('/content/drive/MyDrive/Project/Data/datasets/' + name)
table = table[[colname for colname in table.columns if 'Unnamed:' not in colname]]

In [6]:
scraped_path = '/content/drive/MyDrive/Project/Data/Scraped/'

if not path.exists(scraped_path):
  mkdir(scraped_path)

scraped_filename = scraped_path + name[:name.find('.csv')] + '_scraped.csv'

old_cols = table.columns.to_list() 

scrape_cols = ['image', 'has_video', 'n_tiers', 'tiers_values', 'n_images', ' n_gifs', 
                             'n_websites', 'fb_linked', 'n_collab', 'collab_names']
cols = old_cols + scrape_cols
df = pd.DataFrame(columns=cols)

if path.exists(scraped_filename):
  df = pd.read_csv(scraped_filename)
  # cut table to restart after last index of previous run
  table = table[df.index.stop:]

# Scraping

In [ ]:
http_proxy  = None
https_proxy = None
proxy = None
proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
            }
start = time.time()
k = False
fs = FuturesSession()
sess = Session()
r = fs.get("https://www.kickstarter.com").result()
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]
headers= {
              "x-csrf-token": xcsrf
          }
query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""
 
last_10 = []
c= 0

for i, row in table.iterrows():
  url = eval(row['urls'])['web']['project']
  succesful = False
  page = time.time()
  slug = re.search('/projects/(.*)\?', url).group(1)
  url = re.search('(.*)\?', url).group(1)
  print(f"------ Page {i}: {slug} ------")
  
  while not succesful:
    try:
      rs = [fs.get(url), fs.post("https://www.kickstarter.com/graph", proxies = proxyDict,
          headers=headers,
          json = {
              "operationName":"Campaign",
              "variables":{
                  "slug": slug
              },
              "query": query
          })]
 
      time.sleep(1.5)
      r = None
      r = sess.get(url + '/creator_bio')
 
      soup = BeautifulSoup(r.content, 'lxml')
      ## Websites
      websites = 0
      if soup.find("ul", {'class':'links list f5 bold'}):
        websites = len(soup.find("ul", {'class':'links list f5 bold'}).find_all('li'))
          
      ## Bio
      bio = soup.find('div', {'class': 'readability'})
      if bio:
        bio_text = []
        for p in bio.find_all('p'):
            if p.find(text=True) != None:
                bio_text.append(p.find(text=True))
            
        bio_text = ' '.join(bio_text)
      else:
        bio_text = None
      ## Facebook
      fb = soup.find("div", {'class':"facebook py2 border-bottom f5"}) 
      fb_linked = fb.find("a", {'class': 'popup'}) != None
      
      # Collaborators
      n_collab = 0
      collaborators = None
      if soup.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}):
        collaborators = soup.find("div", {'class': 'pt3 pt7-sm mobile-hide row'}).findChildren('a')
        n_collab = len(collaborators)
      
      # names
      collab_names = []
      if collaborators:
        for col in collaborators:
            collab_names.append(re.search(r'/profile/(.*?)/about', str(col)).group(1))   
 
      r = None
      r = rs[0].result()
      soup = BeautifulSoup(r.content, 'lxml')
      
      ## Image
      image = soup.find('img')['src']
      ## Video
      has_video = soup.find('video') is not None
      ## Pledge tiers
      tiers = soup.find_all("div", {"class": "pledge__info"})
      tiers_values = []
      for tier in tiers:
          s = str(tier.find("span", {"class": "money"}))
          if re.search(r'\d+', s):
            tiers_values.append(int(re.findall(r'\d+', s)[0]))
          else:
            tiers_values.append('0')
      n_tiers = len(tiers_values)
 
      # time.sleep(random.lognormal(mean = 0.1, sigma=0.25))
      r = None
      r = rs[1].result()
      result = r.json()
      
      story_html = result["data"]["project"]["story"]
      story = BeautifulSoup(story_html, 'html.parser')
      n_gifs = len(story.find_all('img', {'class': "fit js-lazy-image"}))
      n_images = len(story.find_all('img')) - n_gifs
      
 
      # text
      story_text = ' '.join([p for p in story.find_all(text=True) if i not in ['\n', ' ']])
 
      risks = result["data"]["project"]["risks"]
 
      # time.sleep(random.lognormal(mean = 0.1, sigma=0.25))
      
 
      df.loc[i] = pd.Series(table.loc[i].values.tolist() + [image, has_video, n_tiers, tiers_values, n_images, n_gifs, websites,
                      fb_linked, n_collab, collab_names], index = cols)
      succesful = True
      if time.time()-page < 2:
        time.sleep(2 - (time.time() - page))
      print('Time for this page was {}s'.format(round(time.time()-page, 2)))
      
      last_10.append(time.time()-page)
      if len(last_10) == 11:
        last_10.pop(0)
        if sum([x >= 5 for x in last_10]) >= 5:
          print('Proxy too slow')
          proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
          if len(proxy_list) == 0:
            req_proxy = RequestProxy()
            proxy_list = req_proxy.get_proxy_list()
          http_proxy  = 'http://' + proxy
          https_proxy = 'https://' + proxy
          proxyDict = { 
                        "http"  : http_proxy, 
                        "https" : https_proxy, 
                      }
         
          last_10 = []
 
    except KeyboardInterrupt:
      k = True
      print('interrupted!')
      break
    
    except:
      if r and r.status_code == 200:
        time.sleep(1.5)
        print('Problems parsing, scraping skipped!!')
        df.loc[i] = pd.Series(table.loc[i].values.tolist(), index = old_cols)
        succesful =True
      elif r and r.status_code == 429:
        print("Too many requests, rotate ip")
        time.sleep(5)
        proxy = proxy_list.pop(random.choice(len(good_proxies))).get_address()
        if len(proxy_list) == 0:
            req_proxy = RequestProxy()
            proxy_list = req_proxy.get_proxy_list()
        http_proxy  = 'http://' + proxy
        https_proxy = 'https://' + proxy
        proxyDict = { 
                      "http"  : http_proxy, 
                      "https" : https_proxy, 
                    }
   
      else:
        print("Bad Proxy")
        time.sleep(5)
        proxy = proxy_list.pop(random.choice(len(proxy_list))).get_address()
        if len(proxy_list) == 0:
            req_proxy = RequestProxy()
            proxy_list = req_proxy.get_proxy_list()
        http_proxy  = 'http://' + proxy
        https_proxy = 'https://' + proxy
        proxyDict = { 
                      "http"  : http_proxy, 
                      "https" : https_proxy, 
                    }
        
    
    
  if k:
    break
  
  if i%500 == 0:
    print(f'\n\n ----- Reached Page {i}, saving dataframe to {scraped_filename} ----- \n\n')
    df.to_csv(scraped_filename, index=False)
 
  
print(f'Total time for {i - table.index.start} pages is {round(time.time()-start, 2)} seconds')

------ Page 10215: 1692006165/the-wall-americas-separation-of-church-and-state ------
Time for this page was 2.0s
------ Page 10216: thekobiechronicles/from-fatherless-to-fatherhood ------
Time for this page was 2.0s
------ Page 10217: ericatrembay/it-came-from-above-the-joplin-mo-tornado ------
Time for this page was 2.19s
------ Page 10218: ethanmarten/white-buffalo-an-american-prophecy-lakota-and-hopi ------
Time for this page was 2.05s
------ Page 10219: eastpointpictures/the-smash-brothers-series-production ------
Time for this page was 2.42s
------ Page 10220: btivie/the-drop-box ------
Time for this page was 2.32s
------ Page 10221: justinzimmerman/excalibur-comics-women-of-wonder-2011-documentary ------
Time for this page was 2.04s
------ Page 10222: orlive/hiv-a-whole-different-story-or-live ------
Time for this page was 2.0s
------ Page 10223: thehappyfilm/the-happy-film ------
Time for this page was 2.04s
------ Page 10224: theclubdoc/the-club-0 ------
Time for this page was

In [ ]:
df.to_csv(scraped_filename, index=False)